In [1]:
import os
import numpy as np
import glob
from astropy.timeseries import LombScargle
from tkinter import simpledialog
from tkinter import messagebox
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import tkinter as tk
import matplotlib.image as mpimg

from PIL import Image, ImageTk
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg



In [2]:

Tiempos=[]
Magnitudes=[]
Errores=[]
Tiempo=[]
Magnitud=[]
Error=[]
T_i=0.008
T_f=20
i_star=0
pico_seleccionado=None
picos=None
TipoEstrella=-1
Ymax=-1
Ymin=-1
archivos=[]
def crear_carpeta(nombre_carpeta):
    ruta_carpeta = os.path.join(os.getcwd(), nombre_carpeta)
    if not os.path.exists(ruta_carpeta):
        os.makedirs(ruta_carpeta)
        
def cargar_archivo():
    carpeta_seleccionada = filedialog.askdirectory()
    global Tiempos
    global Magnitudes
    global Errores
    global archivos
    global i_star
    if carpeta_seleccionada:
        archivos = [archivo for archivo in os.listdir(carpeta_seleccionada) if archivo.endswith(".txt")]
        for archivo in archivos:

            ruta_completa = os.path.join(carpeta_seleccionada, archivo)

            try:
                Datos = np.loadtxt(ruta_completa)
                Tiempos.append(Datos.T[:,0])
                Magnitudes.append(Datos.T[:,1])
                Errores.append(Datos.T[:,2])
            except Exception as e:
                print("Error al leer el archivo:", archivo)
    global Magnitud
    global Error
    global Tiempo
    global T_i
    global T_f
    
    i_star=0
    Tiempo=Tiempos[i_star]
  
    Magnitud=Magnitudes[i_star]
    Error=Errores[i_star]
    print(len(Tiempo),len(Magnitud),len(Error))
    actualizarperiodograma()
    T_i=0.008
    T_f=25
    label2.config(text=str(archivos[i_star]).split(".")[0])
    
    crear_carpeta("Cefeida")
    crear_carpeta("RRLyrae")
    crear_carpeta("No_clasificada")
    crear_carpeta("Algol")


In [3]:
def periodograma(T_i,T_f,n=100000):
    frecuency=np.linspace(1/T_f,(1/T_i),1000000)
    power = LombScargle(Tiempo, Magnitud, Error).power(frecuency)
    max_power = power.max()
    fap = LombScargle(Tiempo, Magnitud, Error).false_alarm_probability(max_power)
    periodos=1/np.array(frecuency)
   
    
    return(frecuency,power,periodos,fap)

def actualizarperiodograma():
    
    x,y,z,f=periodograma(T_i,T_f)
    global picos
    ax.clear()
    ax.plot(z,y)
    ax.set_ylabel("Power")
    ax.set_xlabel("Periodo (dias)")
    ax.axhline(f,color="r",linestyle="--")
    power=y
    frecuency=x
    peaks, _ = find_peaks(power)
    powerpeaks=power[peaks]
    power_order=np.sort(powerpeaks)[::-1]
    n_peaks=5
    ind_peaks=[]
    Freq_peaks=[]
    Power_peaks=[]
    Periodo_peaks=[]
    i=0
    colores =  ['orange', 'cyan', '#FF00FF', 'lime', 'dodgerblue']
    while(i<n_peaks):
        ind_peaks=np.where(power==power_order[i])[0][0]
        Freq_peaks.append(frecuency[ind_peaks])
        Periodo_peaks.append(1/Freq_peaks[-1])
        Power_peaks.append(power[ind_peaks])
        i=i+1
    picos=Periodo_peaks
    ax.scatter(1/np.array(Freq_peaks),Power_peaks,c=colores)
    canvas.draw()
    
def Periodo_Mayor():
    global T_f
    texto_ingresado = simpledialog.askstring("Periodo Mayor", "Por favor ingresa el periodo mayor:")
    if texto_ingresado:
        T_f=float(texto_ingresado)
    actualizarperiodograma()
    
def Limite():
    global Ymax
    global Ymin
    texto_ingresado = simpledialog.askstring("Limite Mayor", "Por favor ingresa el Limite mayor:")
    if texto_ingresado:
        Ymax=float(texto_ingresado)
    texto_ingresado = simpledialog.askstring("Limite Menor", "Por favor ingresa el Limite menor:")
    if texto_ingresado:
        Ymin=float(texto_ingresado)
    ax2.set_ylim(Ymin,Ymax)
    canvas2.draw()

def Periodo_Menor():
    global T_i
    texto_ingresado = simpledialog.askstring("Periodo Menor", "Por favor ingresa el periodo menor:")
    if texto_ingresado:
        T_i=float(texto_ingresado)
    actualizarperiodograma()

def seleccionar_pico(pico):
    pico=pico-1
    global pico_seleccionado
    pico_seleccionado=picos[pico]
    curva_luz()
    
def seleccionar_checkbox(checkbox, var,n):
    if var.get() == 1:
        checkboxes = [pico1_checkbox, pico2_checkbox, pico3_checkbox, pico4_checkbox, pico5_checkbox]
        vars = [pico1_var, pico2_var, pico3_var, pico4_var, pico5_var]
        for cb, v in zip(checkboxes, vars):
            if cb is not checkbox:
                v.set(0)
        seleccionar_pico(n)

In [4]:
#Definicion de la funcion de fase
def phi(t,t0,p):
    return (((t-t0)/p)-int((t-t0)/p))
def curva_luz():
    
    ax2.clear()
    Tie = [phi(i, Tiempo[0], pico_seleccionado) for i in Tiempo]
    Tie=np.array(Tie)
    valor_formateado = "{:.4f}".format(pico_seleccionado)
    ax2.scatter(Tie, Magnitud,c="r",s=6)
    ax2.scatter(Tie+1, Magnitud,c="r",s=6)
    ax2.scatter(Tie-1, Magnitud,c="r",s=6)
    ax2.set_title('Periodo ' + str(valor_formateado) + " dias")
    ax2.set_xlabel("Fase")
    ax2.set_ylabel("Magnitud")
    ax2.invert_yaxis()  # Voltea el eje y
    
    canvas2.draw()

def dividir_cuatro():
    global pico_seleccionado
    pico_seleccionado=pico_seleccionado/4
    curva_luz()

def dividir_dos():
    global pico_seleccionado
    pico_seleccionado=pico_seleccionado/2
    curva_luz()

def multiplicar_dos():
    global pico_seleccionado
    pico_seleccionado=pico_seleccionado*2
    curva_luz()

def multiplicar_cuatro():
    global pico_seleccionado
    pico_seleccionado=pico_seleccionado*4
    curva_luz()
    


In [5]:
def desmarcar_checkboxes():
    check_binaria_var.set(0)
    check_cefeida_var.set(0)
    check_NC_var.set(0)
    check_rrly_var.set(0)
    pico1_var.set(0)
    pico2_var.set(0)
    pico3_var.set(0)
    pico4_var.set(0)
    pico5_var.set(0)


def seleccionar_checkbox_tipo(checkbox, var,n):
    if var.get() == 1:
        checkboxes = [ckeck_cefeida,ckeck_binaria,ckeck_NC,ckeck_rrly]
        vars = [check_cefeida_var, check_binaria_var, check_NC_var, check_rrly_var]
        for cb, v in zip(checkboxes, vars):
            if cb is not checkbox:
                v.set(0)
        global TipoEstrella
        TipoEstrella =n
def mostrar_cuadro_texto(mensaje):
    messagebox.showinfo("Mensaje", mensaje)
    
def guardar():
    Tipos=["Cefeida","RRLyrae","Algol","No_clasificada"]
    global Tiempo
    global Magnitud
    global Error
    global i_star
    global TipoEstrella
    global archivos
    global T_i
    global T_f
    global Ymin
    global Ymax
    global pico_seleccionado
    if(TipoEstrella>=0):
        fig_N,ax_N= plt.subplots(figsize=(15, 6))
        Tie = [phi(i, Tiempo[0], pico_seleccionado) for i in Tiempo]
        Tie=np.array(Tie)
        ax_N.scatter(Tie,Magnitud,c="navy",label=f"Período [dias]={pico_seleccionado:.3f}")
        ax_N.scatter(Tie+1,Magnitud,c="navy")
        ax_N.scatter(Tie-1,Magnitud,c="navy")
        ax_N.set_ylabel("Magnitud",fontsize=22)
        ax_N.set_xlabel("Fase",fontsize=22)
        plt.legend(fontsize=16,loc="upper right")
        ax_N.invert_yaxis()
        L=archivos[i_star].split(".")[0].split("_")[1]
        ax_N.set_title(str(L),fontsize=28,y=1.05)
        Name_dic=Tipos[TipoEstrella]
        ruta_carpeta = os.path.join(os.getcwd(), Name_dic)
        ruta_guardar= os.path.join(ruta_carpeta, str(archivos[i_star]))
        X=np.array([Tiempos[i_star],Magnitudes[i_star],Errores[i_star]])
        np.savetxt(ruta_guardar,X)
        name="CurvaLuz"+str(archivos[i_star]).split(".")[0]+".png"
        namefig= os.path.join(ruta_carpeta, name)
        if(Ymax!=-1):
            ax_N.set_ylim(Ymin,Ymax)
        fig_N.savefig(namefig)
        Ymin=-1
        Ymax=-1
        i_star=i_star+1
        if(i_star<len(archivos)):
            Tiempo=Tiempos[i_star]
            Magnitud=Magnitudes[i_star]
            Error=Errores[i_star]
            mostrar_cuadro_texto("Estrella: "+str(i_star-1)+" ,guardada como: "+str(Name_dic))
      
            T_i=0.008
            T_f=25
            label2.config(text=str(archivos[i_star]).split(".")[0])
            actualizarperiodograma()
            
            ax2.clear()
            canvas2.draw()
        else:
            mostrar_cuadro_texto("Estrellas Terminadas")
            ax2.clear()
            canvas2.draw()
            ax.clear()
            canvas.draw()
            label2.config(text="Finalizada")
        desmarcar_checkboxes()
    else:
        mostrar_cuadro_texto("No se ha seleccionado estrellas")
    
def descartar():
    global i_star
    global archivos
    global T_i
    global T_f
    global Tiempo
    global Magnitud
    global Error
    i_star=i_star+1
    if(i_star<len(archivos)):
        Tiempo=Tiempos[i_star]
        Magnitud=Magnitudes[i_star]
        Error=Errores[i_star]
        T_i=0.08
        T_f=25
        actualizarperiodograma()
        label2.config(text=str(archivos[i_star]).split(".")[0])
        mostrar_cuadro_texto("Estrella Descartada"+str(i_star))
        ax2.clear()
        canvas2.draw()
        
    else:
        mostrar_cuadro_texto("Estrellas Terminadas")
        ax2.clear()
        canvas2.draw()
        ax.clear()
        canvas.draw()
        label2.config(text="Finalizada")
    desmarcar_checkboxes()

In [6]:
def abrir_ventana_graficas():
    ventana_graficas = tk.Toplevel(root)
    ventana_graficas.title("Gráficas")

    # Crear una matriz de 4x5 subgráficas
    fig_Ventana, axs_ven = plt.subplots(5, 5, figsize=(15, 10), sharex=True, sharey=True)
    
    global picos
    global Tiempo
    global Magnitud
    Multiplier=[0.25,0.5,1,2,4]
    colores =  ['orange', 'cyan', '#FF00FF', 'lime', 'dodgerblue']
    for i in range(5):
        for j in range(5):

            Tie = [phi(ink, Tiempo[0], picos[j]*Multiplier[i]) for ink in Tiempo]
            Tie=np.array(Tie)
            axs_ven[i, j].scatter(Tie, Magnitud,c=colores[j],s=6)
            axs_ven[i, j].scatter(Tie+1, Magnitud,c=colores[j],s=6)
            axs_ven[i, j].scatter(Tie-1, Magnitud,c=colores[j],s=6)
            axs_ven[i,j].invert_yaxis()

    
    for j in range(5):
        axs_ven[0, j].set_title(f'Pico {j + 1}')

    Fila=["T/4","T/2","T","T*2","T*4"]
    for i in range(5):
        axs_ven[i, 0].set_ylabel(Fila[i], rotation=90, labelpad=15)

    # Añadir la figura a la ventana de Tkinter
    canvas = FigureCanvasTkAgg(fig_Ventana, master=ventana_graficas)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    # Botón para cerrar la ventana
    boton_cerrar = tk.Button(ventana_graficas, text="Cerrar", command=ventana_graficas.destroy)
    boton_cerrar.pack(side=tk.BOTTOM)


In [ ]:

# Crear la ventana principal
root = tk.Tk()
root.title("Busqueda de estrellas")
# Frame 1
framePrincipal = tk.Frame(root, borderwidth=2, relief="ridge", width=500, height=500)
framePrincipal.grid(row=0, column=0, padx=0, pady=0)


frame1 = tk.Frame(framePrincipal, borderwidth=2, relief="ridge", width=400, height=50)
frame1.grid(row=0, column=0, padx=10, pady=10)

button = tk.Button(frame1, text="Cargar Archivo", command=cargar_archivo)
button.grid(row=0,column=0,padx=10,pady=10)

label1 = tk.Label(frame1, text="Estrella Actual ")
label1.grid(row=0,column=1,padx=10,pady=10)
label2 = tk.Label(frame1, text="--")
label2.grid(row=0,column=2,padx=10,pady=10)



# Frame 2
frame2 = tk.Frame(framePrincipal, borderwidth=2, relief="ridge", width=450, height=500)
frame2.grid(row=1, column=0, padx=0, pady=0)

frame_fig=tk.Frame(frame2, borderwidth=2, relief="ridge", width=450, height=420)
frame_fig.grid(row=0, column=0, padx=0, pady=0)

fig, ax = plt.subplots(figsize=(4,2))

# Crear un lienzo de Matplotlib dentro del Frame
canvas = FigureCanvasTkAgg(fig, master=frame_fig)
canvas.draw()
canvas.get_tk_widget().pack()

frame_butto =tk.Frame(frame2, borderwidth=2, relief="ridge", width=450, height=80)
frame_butto.grid(row=1, column=0, padx=0, pady=0)

frame_periodos=tk.Frame(frame_butto, borderwidth=2, relief="ridge", width=250, height=100)
frame_periodos.grid(row=0,column=0,padx=0,pady=0)
boton_periodoM = tk.Button(frame_periodos, text="T mayor", command=Periodo_Mayor)
boton_periodoM.grid(row=0,column=0,padx=2, pady=2)


boton_periodom = tk.Button(frame_periodos, text="T menor", command=Periodo_Menor)
boton_periodom.grid(row=0,column=1,padx=2, pady=2)

colores =  ['orange', 'cyan', '#FF00FF', 'lime', 'dodgerblue']


frame_picos=tk.Frame(frame_butto, borderwidth=2, relief="ridge", width=200, height=100)
frame_picos.grid(row=0,column=1,padx=0,pady=0)

pico1_var = tk.IntVar()
pico1_checkbox = tk.Checkbutton(frame_picos, text="Pico 1",fg=colores[0], variable=pico1_var, command=lambda: seleccionar_checkbox(pico1_checkbox, pico1_var,1))
pico1_checkbox.grid(row=0,column=0,padx=0,pady=0)

pico2_var = tk.IntVar()
pico2_checkbox = tk.Checkbutton(frame_picos, text="Pico 2",fg=colores[1], variable=pico2_var, command=lambda: seleccionar_checkbox(pico2_checkbox, pico2_var,2))
pico2_checkbox.grid(row=0,column=1,padx=0,pady=0)

pico3_var = tk.IntVar()
pico3_checkbox = tk.Checkbutton(frame_picos, text="Pico 3",fg=colores[2], variable=pico3_var, command=lambda: seleccionar_checkbox(pico3_checkbox, pico3_var,3))
pico3_checkbox.grid(row=0,column=2,padx=0,pady=0)


pico4_var = tk.IntVar()
pico4_checkbox = tk.Checkbutton(frame_picos, text="Pico 4",fg=colores[3], variable=pico4_var, command=lambda: seleccionar_checkbox(pico4_checkbox, pico4_var,4))
pico4_checkbox.grid(row=0,column=3,padx=0,pady=0)

pico5_var = tk.IntVar()
pico5_checkbox = tk.Checkbutton(frame_picos, text="Pico 5",fg=colores[4], variable=pico5_var, command=lambda: seleccionar_checkbox(pico5_checkbox, pico5_var,5))
pico5_checkbox.grid(row=0,column=4,padx=0,pady=0)
# Frame 3
frame3 = tk.Frame(framePrincipal, borderwidth=2, relief="ridge", width=300, height=150)
frame3.grid(row=2, column=0, padx=10, pady=10)

frame_curva = tk.Frame(frame3, borderwidth=2, relief="ridge", width=300, height=100)
frame_curva.grid(row=0, column=0, padx=10, pady=10)


frame_curva_bot = tk.Frame(frame3, borderwidth=2, relief="ridge", width=300, height=50)
frame_curva_bot.grid(row=1, column=0, padx=10, pady=10)

# Crear botones para las operaciones
boton_dividir_cuatro = tk.Button(frame_curva_bot, text="T/4", command=dividir_cuatro)
boton_dividir_cuatro.grid(row=0, column=0,padx=10, pady=5)

boton_dividir_dos = tk.Button(frame_curva_bot, text="T/2", command=dividir_dos)
boton_dividir_dos.grid(row=0, column=1,padx=10, pady=5)

boton_multiplicar_dos = tk.Button(frame_curva_bot, text="Tx2", command=multiplicar_dos)
boton_multiplicar_dos.grid(row=0, column=2,padx=10, pady=5)

boton_multiplicar_cuatro = tk.Button(frame_curva_bot, text="Tx4", command=multiplicar_cuatro)
boton_multiplicar_cuatro.grid(row=0, column=3,padx=10, pady=5)

boton_figures= tk.Button(frame_curva_bot,text="Damian",command=abrir_ventana_graficas)
boton_figures.grid(row=0,column=4,padx=10,pady=5)


boton_X = tk.Button(frame_curva_bot, text="Limite", command=Limite)
boton_X.grid(row=0,column=5,padx=2, pady=2)

fig2, ax2 = plt.subplots(figsize=(4,2))

# Crear un lienzo de Matplotlib dentro del Frame
canvas2 = FigureCanvasTkAgg(fig2, master=frame_curva)
canvas2.draw()
canvas2.get_tk_widget().pack()
#Ultimo Frame

frame4 = tk.Frame(framePrincipal, borderwidth=2, relief="ridge", width=500, height=50)
frame4.grid(row=3, column=0, padx=0, pady=0)

frame_checks = tk.Frame(frame4, borderwidth=2, relief="ridge", width=300, height=50)
frame_checks.grid(row=0, column=0, padx=0, pady=0)

check_cefeida_var=tk.IntVar()
ckeck_cefeida = tk.Checkbutton(frame_checks, text="Cefeida", variable=check_cefeida_var,command=lambda: seleccionar_checkbox_tipo(ckeck_cefeida, check_cefeida_var,0))
ckeck_cefeida.grid(row=0, column=0, padx=1, pady=1)

check_rrly_var=tk.IntVar()
ckeck_rrly = tk.Checkbutton(frame_checks, text="RRLyrae", variable=check_rrly_var,command=lambda: seleccionar_checkbox_tipo(ckeck_rrly, check_rrly_var,1))
ckeck_rrly.grid(row=0, column=2, padx=0, pady=0)

check_binaria_var=tk.IntVar()
ckeck_binaria = tk.Checkbutton(frame_checks, text="Algol", variable=check_binaria_var,command=lambda: seleccionar_checkbox_tipo(ckeck_binaria, check_binaria_var,2))
ckeck_binaria.grid(row=0, column=3, padx=0, pady=0)

check_NC_var=tk.IntVar()
ckeck_NC = tk.Checkbutton(frame_checks, text="No Clasificada", variable=check_NC_var,command=lambda: seleccionar_checkbox_tipo(ckeck_NC, check_NC_var,3))
ckeck_NC.grid(row=0, column=4, padx=0, pady=0)

    
frame_but = tk.Frame(frame4, borderwidth=2, relief="ridge", width=200, height=50)
frame_but.grid(row=0, column=1, padx=0, pady=0)
# Ejecutar el bucle de eventos

# Botón 1
botonguardar = tk.Button(frame_but, text="Guardar", command=guardar)
botonguardar.grid(row=0, column=0, sticky="w")

# Botón 2
botondescartar = tk.Button(frame_but, text="Descartar", command=descartar)
botondescartar.grid(row=0, column=1, sticky="w")

frameEjemplos = tk.Frame(root, borderwidth=2, relief="ridge", width=200, height=500)
frameEjemplos.grid(row=0, column=1, padx=0, pady=0)

frameCefeida = tk.Frame(frameEjemplos, borderwidth=2, relief="ridge", width=200, height=200)
frameCefeida.grid(row=0, column=0, padx=0, pady=0)

frameCefeida_T = tk.Frame(frameCefeida, borderwidth=0, relief="ridge", width=200, height=30, bg='lightgray', bd=5)
frameCefeida_T.grid(row=0, column=0, padx=0, pady=0)

label_Cefeida = tk.Label(frameCefeida_T, text="Cefeida tiene T< 1 dia", bg='white')
label_Cefeida.pack(padx=0, pady=0)

frameCefeida_I = tk.Frame(frameCefeida, borderwidth=0, relief="ridge", width=200, height=170)
frameCefeida_I.grid(row=1, column=0, padx=0, pady=0)


frameRR = tk.Frame(frameEjemplos, borderwidth=2, relief="ridge", width=200, height=200)
frameRR.grid(row=1, column=0, padx=0, pady=0)

frameRR_T = tk.Frame(frameRR, borderwidth=0, relief="ridge", width=200, height=30, bg='lightgray', bd=5)
frameRR_T.grid(row=0, column=0, padx=0, pady=0)

label_RR = tk.Label(frameRR_T, text="RR Lyrae tiene T< 1 dia", bg='white')
label_RR.pack(padx=0, pady=0)

frameRR_I = tk.Frame(frameRR, borderwidth=0, relief="ridge", width=200, height=170)
frameRR_I.grid(row=1, column=0, padx=0, pady=0)


frameEA = tk.Frame(frameEjemplos, borderwidth=2, relief="ridge", width=200, height=200)
frameEA.grid(row=2, column=0, padx=0, pady=0)

frameEA_T = tk.Frame(frameEA, borderwidth=0, relief="ridge", width=200, height=30, bg='lightgray', bd=5)
frameEA_T.grid(row=0, column=0, padx=0, pady=0)

label_EA = tk.Label(frameEA_T, text="Binaria Tipo Algol", bg='white')
label_EA.pack(padx=0, pady=0)

frameEA_I = tk.Frame(frameEA, borderwidth=0, relief="ridge", width=200, height=170)
frameEA_I.grid(row=1, column=0, padx=0, pady=0)


image_path_EA = "./Ejemplos/CurvaLuzEA.png"  # Cambia esta ruta por la ubicación de tu imagen
image_path_RR = "./Ejemplos/CurvaLuzRRLyrae.png"
image_path_CEF = "./Ejemplos/CurvaLuzCefeida.png"

imgEA = mpimg.imread(image_path_EA)
imgRR = mpimg.imread(image_path_RR)
imgCe = mpimg.imread(image_path_CEF)

figEA, axEA = plt.subplots(figsize=(2.5,1.5))
figRR, axRR = plt.subplots(figsize=(2.5,1.5))
figCe, axCe = plt.subplots(figsize=(2.5,1.5))

imgplot_EA = axEA.imshow(imgEA)
imgplot_RR = axRR.imshow(imgRR)
imgplot_Ce = axCe.imshow(imgCe)

axEA.axis("off")
axRR.axis("off")
axCe.axis("off")
canvasEA = FigureCanvasTkAgg(figEA, master=frameEA_I)
canvasRR = FigureCanvasTkAgg(figRR, master=frameRR_I)
canvasCe = FigureCanvasTkAgg(figCe, master=frameCefeida_I)

canvasCe.draw()
canvasCe.get_tk_widget().pack()
canvasRR.draw()
canvasRR.get_tk_widget().pack()
canvasEA.draw()
canvasEA.get_tk_widget().pack()

root.mainloop()
